In [ ]:
from sklearn.datasets import load_files
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.decomposition import LatentDirichletAllocation
import nltk
from nltk.cluster import KMeansClusterer
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import string
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import json
from nltk.stem import WordNetLemmatizer 
from eventregistry import *
%matplotlib inline

# 1. Pull from API

In [ ]:
MY_API_KEY = '4a2c424f-0a05-4691-b59b-7b7338b3ba9e'
er = EventRegistry(apiKey = MY_API_KEY)
# q = QueryArticlesIter(conceptUri = er.getConceptUri("Xi Jinping"),lang="eng",dataType="news",sourceUri=QueryItems.OR([
#     er.getNewsSourceUri("nytimes.com"),
#     er.getNewsSourceUri("wsj.com")
# ]))
q = QueryArticlesIter(conceptUri = er.getConceptUri("trade war"),lang="eng",dataType="news",sourceUri="nytimes.com")

articles = []
first = True
bad_prefix = set()
bad_prefix.add("On Politics: The Biggest Stories of the Week")
bad_prefix.add("DealBook Briefing:")
bad_suffix = set()
bad_suffix.add("Briefing")

size = 35
counter = 0
for art in q.execQuery(er, sortBy = "date", maxItems=60):
    if size == counter:
        break
    if len(art['body']) > 3000 and not (art['title'].startswith(tuple(bad_prefix)) or art['title'].endswith(tuple(bad_suffix))):
        counter += 1
        articles.append(art)

df = pd.DataFrame(articles)

# 1a. Pull article manually

In [ ]:
# new_articles = []

In [ ]:
# article = {}
# article['body'] = ""
# article['date'] = "YYYY-MM-DD"
# article['url'] = ""
# article['title'] = ""

# #Deault values
# article['authors'] = []
# article['dataType'] = 'news'
# article['dateTime'] = None
# article['eventUri'] = None
# article['image'] = None
# article['idDuplicate'] = False
# article['lang'] = "eng"
# article['sentiment'] = None
# article['sim'] = None
# article['source'] = None
# article['time'] = None
# article['uri'] = None
# article['wgt'] = None
# new_articles.append(article)

In [ ]:
# df = pd.concat([pd.DataFrame(new_articles), df], ignore_index=True) #places the new articles on top of the articles in the existing dataframe

# 2. Text processing

In [ ]:
lemmatizer = WordNetLemmatizer()
def process_text(text):
    words = nltk.word_tokenize(text)
    words = [w.lower() for w in words]
    words = [w.translate(str.maketrans('', '', string.punctuation)) for w in words]
    words = [w for w in words if w not in nltk.corpus.stopwords.words('english')]
    words = [lemmatizer.lemmatize(w) for w in words]
    words = [w for w in words if w.isalnum()] 
    new_text = " ".join(words)
    return new_text

# 2.a Process "body" of the article

In [ ]:
df['body'] = [process_text(b) for b in df['body']]

# (Optional) Split articles (i.e. old articles vs new article)

In [ ]:
# df_new =df.iloc[:1,:]
# df_old = df.iloc[1:,:]